In [1]:
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.preprocessing import scale
from scipy.stats.mstats import winsorize
from cnv_constraint import overlaps, merge_intervals

In [2]:
# load genes, take subset
with open('/scratch/users/magu/toolbox/glist-hg19', 'r') as g:
    genes = {line.split()[3]:list(map(int,line.split()[:3])) for line in g if line.split()[0] in list(map(str,range(1,23)))}

with open('/oak/stanford/groups/jpriest/magu/gencode.v29.protein_coding_gene_names.txt', 'r') as sublist:
    gene_list = list(set([line.rstrip() for line in sublist]) & set(genes.keys())) # protein coding

making = False
if making:
    # load targets
    with open('/oak/stanford/groups/mrivas/ukbb24983/cal/pgen/ukb24983_cal_cALL_v2.bim', 'r') as bim:
        sites = [[line.split(None,4)[0], int(line.split(None,4)[3]), int(line.split(None,4)[3])+1] for line in bim]

    # load segdups, merge to unique interval list
    with open('/oak/stanford/groups/jpriest/magu/segdup_hg19.bed', 'r') as sd_bed:
        segdups = [line.split(None,3)[:3] for line in sd_bed]

    segdups = [[str(c)]+interval for c in range(1,23) for interval in merge_intervals([
                                [int(segdup[1]),int(segdup[2])] for segdup in segdups if segdup[0]=='chr'+str(c)])]

In [3]:
# load CNVs
if making:
    cnv_ref='/oak/stanford/groups/jpriest/cnv_ukb/all.cnv'
    iid_cnvs, iid_n_cnvs = {}, {}
    with open(cnv_ref, 'r') as f:
        for i,line in enumerate(f):
            if not i:
                continue
            fid,iid,chrom,bp1,bp2,cn,etc=line.split(None,6)
            if iid in iid_cnvs:
                iid_n_cnvs[iid] += 1
                iid_cnvs[iid].append((chrom,int(bp1),int(bp2),int(cn)))
            else:
                iid_n_cnvs[iid] = 1
                iid_cnvs[iid] = [(chrom,int(bp1),int(bp2),int(cn))]

    # filter people out if they have > 30 CNVs
    cnvs = [cnv for iid,their_cnvs in iid_cnvs.items() if iid_n_cnvs[iid] < 31 for cnv in their_cnvs]

In [4]:
data = pd.read_csv('/oak/stanford/groups/jpriest/magu/cnv_assoc/constraint_stats_20181217.tsv', 
                   sep='\t',
                   index_col=0,
                   header=0)
data['combo'] = data[['n_del','n_dup']].sum(axis=1)
data = data.loc[[gene for gene in gene_list if gene in data.index],:]

if making:
    data = pd.DataFrame({
        'n_cnv':[sum([overlaps(cnv,genes[gene],cn=int(cnv[3])) for cnv in cnvs]) for gene in gene_list],
        'gene_length':[genes[gene][2]-genes[gene][1] for gene in gene_list],
        'n_probes':[sum([overlaps(site,genes[gene]) for site in sites]) for gene in gene_list],
        'frac_segdup':[sum([overlaps(segdup,genes[gene]) for segdup in segdups])/float(genes[gene][2]-genes[gene][1]) 
                                                                                                   for gene in gene_list]
    })

data.describe()

,frac_segdup,gene_length,n_cnv,n_del,n_dup,n_probes,combo
count,17375.000000,1.737500e+04,17375.000000,17375.000000,17375.000000,17375.000000,17375.000000
mean,0.060625,6.201235e+04,169.244892,53.696978,83.367424,22.225957,137.064403
std,0.216839,1.253618e+05,1727.842751,669.609963,1201.056608,43.821801,1442.965469
min,0.000000,1.470000e+02,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,8.384000e+03,5.000000,1.000000,1.000000,5.000000,4.000000
50%,0.000000,2.342300e+04,11.000000,4.000000,3.000000,11.000000,9.000000
75%,0.000000,6.272700e+04,32.000000,12.000000,9.000000,22.000000,25.000000
max,1.000000,2.320934e+06,115755.000000,69259.000000,59008.000000,1376.000000,69990.000000


In [5]:
model = smf.ols(formula='n_cnv ~ n_probes + gene_length + frac_segdup', data=data).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  n_cnv   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     182.5
Date:                Wed, 08 May 2019   Prob (F-statistic):          1.66e-116
Time:                        18:38:53   Log-Likelihood:            -1.5391e+05
No. Observations:               17375   AIC:                         3.078e+05
Df Residuals:                   17371   BIC:                         3.079e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      58.4575     15.209      3.844      0.000      28.646      88.269
n_probes        4.2675      0.466      9.150      0.000       3.353       5.182
gene_length    -0.0010      0.000     -6.277      0.000      -0.001      -0.001
frac_segdup  1308.7819     59.736     21.909      0.000    1191.693    1425.871
==============================================================================
Omnibus:                    47292.756   Durbin-Watson:                   1.644
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       1967816156.136
Skew:                          33.795   Prob(JB):                         0.00
Kurtosis:                    1650.291   Cond. No.                     6.49e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.49e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
null_constraint = dict(zip(data.index, -winsorize(model.resid_pearson, limits=(0,0.05))))
print('Top constrained genes:')
print('\n'.join(['\t'.join([k,str(v)]) for k,v in sorted(null_constraint.items(),key=lambda x:-x[1])[:20]]))

Top constrained genes:
BRCA2	3.40203796691
BRCA1	2.5704693992
APC	2.08599527889
ATM	1.24224254636
MSH2	1.24066290123
MLH1	1.22414379662
PMS2	1.00444108361
MSH6	0.956647998559
RB1	0.904827595186
TTN	0.873320545893
SBDS	0.86121260061
SPATA31D1	0.853122583825
CYP3A4	0.845672525548
FAM205A	0.845166713396
HLA-B	0.834399459797
HLA-C	0.831904122217
PABPC3	0.830792105761
OTOP1	0.830013392684
CYP4A11	0.827792080007
KRT16	0.827596800856


In [7]:
with open('output/cnv_constraint_zscores_20190430.tsv', 'w') as o:
    pli = lambda model: [1 if e < 0 else (-r/e > 0)*(-r/e) for e,r in zip(model.fittedvalues, model.resid)]
    outD = sorted(zip(data.index, 
                      winsorize(model.resid_pearson, limits=(0,0.05)), 
                      pli(model)), 
                  key=lambda x:x[1])
    o.write('\n'.join(['\t'.join([i,str(-j),str(k)]) for i,j,k in outD]))

In [8]:
for cnv_kind in ['n_del','n_dup']:
    # overlapping deletion and spanning duplications
    model = smf.ols(formula=cnv_kind +' ~ n_probes + gene_length + frac_segdup', data=data).fit()
    print(model.summary())
    # show top constraint
    null_constraint = dict(zip(data.index, -winsorize(model.resid_pearson, limits=(0,0.05))))
    print('Top constrained genes:')
    print('\n'.join(['\t'.join([k,str(v)]) for k,v in sorted(null_constraint.items(),key=lambda x:-x[1])[:10]]))
    # write to file
    with open('output/cnv_'+cnv_kind[-3:]+'_zscores_20190430.tsv', 'w') as o:
        pli = lambda model: [1 if e < 0 else (-r/e > 0)*(-r/e) for e,r in zip(model.fittedvalues, model.resid)]
        outD = sorted(zip(data.index, 
                          winsorize(model.resid_pearson, limits=(0,0.05)), 
                          pli(model)), 
                      key=lambda x:x[1])
        o.write('\n'.join(['\t'.join([i,str(-j),str(k)]) for i,j,k in outD]))

                            OLS Regression Results                            
Dep. Variable:                  n_del   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     177.5
Date:                Wed, 08 May 2019   Prob (F-statistic):          2.27e-113
Time:                        18:38:54   Log-Likelihood:            -1.3745e+05
No. Observations:               17375   AIC:                         2.749e+05
Df Residuals:                   17371   BIC:                         2.749e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.3304      5.897      0.056      